In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
training_size = 50000
testing_size = 10000

In [ ]:
training_dataset = unpickle('images/cifar-100-python/train')
testing_dataset = unpickle('images/cifar-100-python/test')

In [ ]:
training_images = training_dataset[b'data']
training_labels = training_dataset[b'fine_labels']
testing_images = testing_dataset[b'data']
testing_labels = testing_dataset[b'fine_labels']

In [ ]:
training_images = training_images[:training_size]
training_labels = training_labels[:training_size]
testing_images = testing_images[:testing_size]
testing_labels = testing_labels[:testing_size]

In [ ]:
training_images = training_images.reshape(len(training_images),3,32,32).transpose(0,2,3,1)
training_labels = np.array(training_labels)
training_labels = training_labels.reshape(len(training_images), 1)

testing_images = testing_images.reshape(len(testing_images),3,32,32).transpose(0,2,3,1)
testing_labels = np.array(testing_labels)
testing_labels = testing_labels.reshape(len(testing_images), 1)

In [ ]:
print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [ ]:
h = []
h_test = []

In [ ]:
hog = cv2.HOGDescriptor((32,32), (16,16), (8,8), (8,8),9)

In [ ]:
for x in training_images:
    h.append(hog.compute(x))

for x in testing_images:
    h_test.append(hog.compute(x))

print("hog computed")

hog computed


In [ ]:
h = np.array(h)
h_test = np.array(h_test)
print(h.shape)
print(h_test.shape)

(50000, 324)
(10000, 324)


In [ ]:
x_train = h
y_train = training_labels

x_test = h_test
y_test = testing_labels

In [ ]:
y_train=y_train.reshape(-1,)
y_test=y_test.reshape(-1,)
print(y_test[0])

49


In [ ]:
def knn_distances(xTrain,xTest,k):
    import numpy as np
    distances = -2 * xTrain@xTest.T + np.sum(xTest**2,axis=1) + np.sum(xTrain**2,axis=1)[:, np.newaxis]
    distances[distances < 0] = 0
    distances = distances**.5
    indices = np.argsort(distances, 0) 
    distances = np.sort(distances,0) 
    return indices[0:k,:], distances[0:k,:]

In [ ]:
def knn_predictions(xTrain,yTrain,xTest,k=8):
    import numpy as np
    indices, distances = knn_distances(xTrain,xTest,k)
    yTrain = yTrain.flatten()
    rows, columns = indices.shape
    predictions = list()
    for j in range(columns):
        temp = list()
        for i in range(rows):
            cell = indices[i][j]
            temp.append(yTrain[cell])
        predictions.append(max(temp,key=temp.count))
    predictions=np.array(predictions)
    return predictions

In [ ]:
def knn_accuracy(yTest,predictions):
    x=yTest.flatten()==predictions.flatten()
    grade=np.mean(x)
    return np.round(grade*100,2)

In [ ]:
predictions = knn_predictions(x_train, y_train, x_test,8)
print('Size of Predictions Array:\n', predictions.shape)

Size of Predictions Array:
 (10000,)


In [ ]:
print('Accuracy:',knn_accuracy(predictions,y_test),'%')

Accuracy: 24.74 %


In [ ]:
print(predictions[1])
print(y_test[1])

45
33


In [ ]:
def acc(yTest,predictions):
    correct = 0
    for i in range(10000):
        if (predictions[i]-yTest[i]) == 0:
            correct = correct + 1 
    return (correct / 10000) * 100
        

In [ ]:
print('Accuracy:',acc(predictions,y_test),'%')

Accuracy: 24.740000000000002 %
